In [1]:
import pandas as pd

/Users/necatifurkancolak/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_stock_exchange = pd.read_excel('../data/raw/borsa_ist.xlsx')
df_bitcoin = pd.read_excel('../data/raw/btc.xlsx')
df_gold = pd.read_excel('../data/raw/gold_prices.xlsx')
df_gov_bonds = pd.read_excel('../data/raw/gov_bonds.xlsx')
df_hpi = pd.read_excel('../data/raw/hpi.xlsx')
df_cpi = pd.read_excel('../data/raw/Turkish Inflation 2018-2024.xlsx')
df_inf = pd.read_excel('../data/raw/Turkish Inflation 2018-2024.xlsx')

# Stock_Exchange:

In [3]:
df_stock_exchange = df_stock_exchange.iloc[1:].reset_index(drop=True)
df_stock_exchange.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_stock_exchange.rename(columns={'Unnamed: 1': 'Price_Stock_Ex'}, inplace=True)
df_stock_exchange['Date'] = pd.to_datetime(df_stock_exchange['Date'], errors='coerce')
df_stock_exchange['Date'] = df_stock_exchange['Date'].dt.strftime('%Y-%m')

df_stock_exchange['stock_return_mo'] = df_stock_exchange['Price_Stock_Ex'] / df_stock_exchange['Price_Stock_Ex'].shift(-1) - 1 #calculate 1 month stock return
df_stock_exchange['stock_return_pa'] = (1 + df_stock_exchange['stock_return_mo'])**(12) - 1 #calculate stock return per annum

df_stock_exchange
df_stock_exchange.to_csv('../data/processed/df_stock_exchange.csv', index=False)

# Bitcoin:

In [4]:
df_bitcoin = df_bitcoin.iloc[1:].reset_index(drop=True)
df_bitcoin.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_bitcoin.rename(columns={'Unnamed: 1': 'Price_Bitcoin'}, inplace=True)
df_bitcoin['Date'] = pd.to_datetime(df_bitcoin['Date'], errors='coerce')
df_bitcoin['Date'] = df_bitcoin['Date'].dt.strftime('%Y-%m')

df_bitcoin['btc_return_mo'] = df_bitcoin['Price_Bitcoin'] / df_bitcoin['Price_Bitcoin'].shift(-1) -1 # calculate 1 month return
df_bitcoin['btc_return_pa'] = (1 + df_bitcoin['btc_return_mo'])**(12) - 1 #calculate return per annum

df_bitcoin
df_bitcoin.to_csv('../data/processed/df_bitcoin.csv', index=False)

# Gold:

In [5]:
df_gold = df_gold.iloc[1:].reset_index(drop=True)
df_gold.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_gold.rename(columns={'Unnamed: 1': 'Price_Gold'}, inplace=True)
df_gold['Date'] = pd.to_datetime(df_gold['Date'], errors='coerce')
df_gold['Date'] = df_gold['Date'].dt.strftime('%Y-%m')

df_gold['gold_return_mo'] = df_gold['Price_Gold'] / df_gold['Price_Gold'].shift(-1) -1 # calculate 1 month return
df_gold['gold_return_pa'] = (1 + df_gold['gold_return_mo'])**(12) - 1 #calculate return per annum (annualized return)

df_gold
df_gold.to_csv('../data/processed/df_gold.csv', index=False)

# Government Bonds:

In [6]:
df_gov_bonds = df_gov_bonds.iloc[1:].reset_index(drop=True)
df_gov_bonds.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_gov_bonds.rename(columns={'Yield (%)': 'Gov_Bond_Yield'}, inplace=True)
df_gov_bonds['Date'] = pd.to_datetime(df_gov_bonds['Date'], errors='coerce')
df_gov_bonds['Date'] = df_gov_bonds['Date'].dt.strftime('%Y-%m')  

df_gov_bonds['bond_return_pa'] = df_gov_bonds['Gov_Bond_Yield'] / 100

df_gov_bonds.to_csv('../data/processed/df_gov_bonds.csv', index=False)

# House Price Index:

In [7]:
df_hpi = df_hpi.iloc[1:].reset_index(drop=True)

df_hpi.rename(columns={'Tarih': 'Date'}, inplace=True)
df_hpi['Date'] = pd.to_datetime(df_hpi['Date'], errors='coerce')
df_hpi['Date'] = df_hpi['Date'].dt.strftime('%Y-%m')

df_hpi = df_hpi.dropna(subset=['Date'])

df_hpi['hpi_return_mo'] = df_hpi['TP KFE TR-1'] / 100
df_hpi['hpi_return_pa'] = (1 + df_hpi['hpi_return_mo'])**(12) - 1 #calculate annualized return

#df_hpi
df_hpi.to_csv('../data/processed/df_hpi.csv', index=False)

# Consumer Price Index

In [8]:
df_cpi = df_cpi.iloc[2:].reset_index(drop=True)
df_cpi.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_cpi.rename(columns={'Unnamed: 1': 'Inflation'}, inplace=True)
df_cpi['Date'] = pd.to_datetime(df_cpi['Date'], errors='coerce', dayfirst=True)
df_cpi['Date'] = df_cpi['Date'].dt.strftime('%Y-%m') 
df_cpi['Inflation'] = df_cpi['Inflation'] / 100 
df_cpi.to_csv('../data/processed/df_cpi.csv', index=False)

# The Adjustment for Annual Returns (for Bitcoin and Gold):

In [9]:
def real_return(nom_ret, inf_rate):
    return ((1 + nom_ret) / (1 + inf_rate)) - 1

merged_df_btc = pd.merge(df_bitcoin, df_cpi, on="Date", how="inner")

merged_df_btc["real_return_bitcoin"] = real_return(merged_df_btc["btc_return_pa"], merged_df_btc["Inflation"] / 100)
merged_df_btc = merged_df_btc.drop(columns=["Inflation"])
merged_df_btc.head()

,Date,Price_Bitcoin,btc_return_mo,btc_return_pa,real_return_bitcoin
0,2024-09,63785.09,0.081934,1.572833,1.560262
1,2024-08,58954.67,-0.086823,-0.663752,-0.665477
2,2024-07,64559.96,0.042888,0.655212,0.645302
3,2024-06,61904.96,-0.084658,-0.65406,-0.656513
4,2024-05,67630.41,0.129642,3.318071,3.285935


In [10]:
merged_df_gold = pd.merge(df_gold, df_cpi, on="Date", how="inner")

merged_df_gold["real_return_gold"] = real_return(merged_df_gold["gold_return_pa"], merged_df_gold["Inflation"] / 100)
merged_df_gold = merged_df_gold.drop(columns=["Inflation"])
merged_df_gold.head()

,Date,Price_Gold,gold_return_mo,gold_return_pa,real_return_gold
0,2024-09,2634.58,0.052405,0.845842,0.836823
1,2024-08,2503.39,0.022794,0.31056,0.303837
2,2024-07,2447.6,0.051939,0.836068,0.825075
3,2024-06,2326.75,-0.000249,-0.002986,-0.010056
4,2024-05,2327.33,0.017968,0.238258,0.229042


# Master_Table:


In [11]:
merged_all = pd.merge(merged_df_btc, merged_df_gold, on="Date", how="inner")
merged_all = pd.merge(merged_all, df_stock_exchange, on="Date", how="inner")
merged_all = pd.merge(merged_all, df_hpi, on="Date", how="inner")
merged_all = pd.merge(merged_all, df_gov_bonds, on="Date", how="inner")
merged_all.head()
merged_all.to_csv('../data/processed/master_table.csv', index=False)

# Database:

In [12]:
import sqlite3

In [13]:
conn = sqlite3.connect('../data/processed/database_name.db')

In [14]:
df_stock_exchange.to_sql('STOCKEX', conn, if_exists='replace', index=False)
merged_df_btc.to_sql('BTC', conn, if_exists='replace', index=False)
merged_df_gold.to_sql('GOLD', conn, if_exists='replace', index=False)
df_gov_bonds.to_sql('BOND', conn, if_exists='replace', index=False)
df_hpi.to_sql("HPI",conn, if_exists='replace', index=False)
df_cpi.to_sql("CPI",conn, if_exists='replace', index=False)

81

In [15]:
query = "SELECT * FROM STOCKEX"
TRIAL = pd.read_sql_query(query, conn)
TRIAL

,Date,Price_Stock_Ex,stock_return_mo,stock_return_pa
0,2024-09,282.62,-0.020619,-0.221210
1,2024-08,288.57,-0.101756,-0.724111
2,2024-07,321.26,-0.012055,-0.135442
3,2024-06,325.18,0.006874,0.085678
4,2024-05,322.96,0.041538,0.629670
...,...,...,...,...
76,2018-05,221.65,-0.134416,-0.823108
77,2018-04,256.07,-0.118793,-0.780752
78,2018-03,290.59,-0.072220,-0.593238
79,2018-02,313.21,-0.014753,-0.163355
